Process the text

In [1]:
import pandas as pd
import re
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Embedding, Conv2D, MaxPooling2D, Flatten, Dropout, Dense, Reshape, Concatenate, Input
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [2]:
# mount the drive where your dataset is available
from google.colab import drive
drive.mount('/content/drive')
filepath='/content/drive/MyDrive/datasets/multimodal_product_classification/' # add your own path. Where to save the dataset

Mounted at /content/drive


In [3]:
# Load data
X_train = pd.read_csv(filepath+'X_train.csv')
y_train = pd.read_csv(filepath+'Y_train.csv')

In [4]:
X_train=X_train.drop(columns="Unnamed: 0")

y_train=y_train.drop(columns="Unnamed: 0")

In [5]:
# Cleaning and Preprocessing Text
def clean_text(text):
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-ZäöüßÄÖÜ ]', '', text)
    # Convert text to lowercase
    text = text.lower()
    return text

In [6]:
# Apply cleaning function to the 'designation' column
X_train['designation'] = X_train['designation'].fillna('').apply(clean_text)

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['designation'])
sequences = tokenizer.texts_to_sequences(X_train['designation'])

# Padding to max length of text
data = pad_sequences(sequences, maxlen=34)

In [7]:
# Assuming the number of unique words in the tokenizer plus 1 is vocab_size
vocab_size = len(tokenizer.word_index) + 1

# Input Layer
input_layer = Input(shape=(34,))

# Embedding Layer
embedding_layer = Embedding(input_dim=vocab_size, output_dim=300, input_length=34)(input_layer)

# Reshape Layer
reshape_layer = Reshape((34, 300, 1))(embedding_layer)

# Conv2D and MaxPooling2D Blocks
conv_blocks = []
for i in range(1, 7):
    # Conv2D Block
    conv = Conv2D(512, (i, 300), activation='relu', padding='valid')(reshape_layer)
    # MaxPooling2D Block
    # Adjusting the pooling size to (1, 1) as the output of Conv2D will have a height of 1
    pool = MaxPooling2D(pool_size=(1, 1))(conv)
    conv_blocks.append(pool)

# Concatenate Layer
concatenate_layer = Concatenate(axis=1)(conv_blocks)

# Flatten Layer
flatten_layer = Flatten()(concatenate_layer)

# Dropout Layer
dropout_layer = Dropout(0.5)(flatten_layer)

# Dense Layer
output_layer = Dense(27, activation='softmax')(dropout_layer)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 34)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 34, 300)              2075670   ['input_1[0][0]']             
                                                          0                                       
                                                                                                  
 reshape (Reshape)           (None, 34, 300, 1)           0         ['embedding[0][0]']           
                                                                                                  
 conv2d (Conv2D)             (None, 34, 1, 512)           154112    ['reshape[0][0]']         

In [8]:
# Split data into training and validation set (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(data, y_train, test_size=0.2, random_state=42)


In [9]:
batch_size=32
epochs=1

In [10]:
# Convert labels to categorical
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Convert to one-hot encoding
y_train_categorical = to_categorical(y_train_encoded)
y_val_categorical = to_categorical(y_val_encoded)

# Adjust the final layer of the model to have as many units as there are unique classes
num_classes = y_train_categorical.shape[1]
model.layers[-1].units = num_classes

# Compile the model again
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train_categorical, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val_categorical))


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


2123/2123 [==============================] - 93s 36ms/step - loss: 0.9714 - accuracy: 0.7226 - val_loss: 0.7349 - val_accuracy: 0.7900


In [11]:
import numpy as np

In [12]:
# Make predictions
predictions = model.predict(X_val)

predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

531/531 [==============================] - 3s 6ms/step


In [13]:
# One-hot encode the validation labels
y_val_categorical = to_categorical(y_val_encoded, num_classes=num_classes)

# Evaluate the model using the one-hot encoded labels
loss, accuracy = model.evaluate(X_val, y_val_categorical)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

531/531 [==============================] - 4s 7ms/step - loss: 0.7349 - accuracy: 0.7900
Test Loss: 0.7348958253860474, Test Accuracy: 0.7899788022041321


In [15]:
y_val_sample

,prdtypecode
61965,1160
73295,2583
51821,1920
6138,1140
65492,1920
56265,50
74970,2905
57651,2060
13358,40
52045,2522


In [17]:
# Selecting the first 10 samples from the validation set
X_val_sample = X_val[0:10]
y_val_sample = y_val[0:10]

# Because the model uses one-hot encoding, convert y_val_sample to categorical
y_val_sample_categorical = to_categorical(label_encoder.transform(y_val_sample))

# Making predictions
predictions = model.predict(X_val_sample)

# Because the model uses one-hot encoding, convert predictions back to label encoding
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

# Comparing actual and predicted labels
for i in range(len(X_val_sample)):
    print(f"Sample {i+1}:")
    t=y_val_sample["prdtypecode"].iloc[i]
    print(f"Actual Label: {t}")
    print(f"Predicted Label: {predicted_labels[i]}\n")


1/1 [==============================] - 0s 17ms/step
Sample 1:
Actual Label: 1160
Predicted Label: 1160

Sample 2:
Actual Label: 2583
Predicted Label: 2583

Sample 3:
Actual Label: 1920
Predicted Label: 1920

Sample 4:
Actual Label: 1140
Predicted Label: 1140

Sample 5:
Actual Label: 1920
Predicted Label: 1920

Sample 6:
Actual Label: 50
Predicted Label: 50

Sample 7:
Actual Label: 2905
Predicted Label: 2905

Sample 8:
Actual Label: 2060
Predicted Label: 2060

Sample 9:
Actual Label: 40
Predicted Label: 1160

Sample 10:
Actual Label: 2522
Predicted Label: 2522

Sample 11:
Actual Label: 2705
Predicted Label: 2705

Sample 12:
Actual Label: 1302
Predicted Label: 1280

Sample 13:
Actual Label: 2522
Predicted Label: 2522

Sample 14:
Actual Label: 2403
Predicted Label: 2403

Sample 15:
Actual Label: 1281
Predicted Label: 2403

Sample 16:
Actual Label: 2403
Predicted Label: 2403

Sample 17:
Actual Label: 2582
Predicted Label: 2582

Sample 18:
Actual Label: 2583
Predicted Label: 2583

Sample 19

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
